In [1]:
import numpy as np
import cv2 # 3.3

In [2]:
cap = cv2.VideoCapture('../../vid/drone-flight.mp4')
ret, frame = cap.read()
print(ret)

True


In [3]:
def run(func, link='../../vid/drone-flight.mp4'):
    cap = cv2.VideoCapture(link)
    while True:
         # Capture frame-by-frame
        ret, frame = cap.read()
        func(ret, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [4]:
def func(ret, frame):
    cv2.imshow('frame' ,frame)
run(func)

In [5]:
def scale(frame, factor=.5):
    height, wid = frame.shape[:2]
    scaled_wid, scaled_height = np.int(wid * factor), np.int(height * factor)
    frame = cv2.resize(frame, (scaled_wid, scaled_height))
    return frame

In [6]:
def orb_func(ret, frame):
    frame = scale(frame)
    orb = cv2.ORB_create()
    kp = orb.detect(frame)
    # kp, des = orb.compute(frame, kp)
    kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    cv2.imshow('kp_drawn', kp_drawn)
run(orb_func)

In [7]:
cv2.__version__

'3.3.0'

In [8]:
# try Kmeans

In [9]:
def get_orb_kp(frame, orb=cv2.ORB_create()):
    frame = scale(frame, .5)
    kp = orb.detect(frame)
    kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    return kp, kp_drawn

In [10]:
def kp_to_pairs(kp, dim):
    max_y, max_x, _ = dim
    out = np.zeros((len(kp), 2))
    for i in range(len(kp)):
        x, y = kp[i].pt
        
        out[i] = np.array([x, max_y-y])
    return out

In [11]:
def get_centers(Z, num=10):
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(Z,num,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    return center

In [12]:
def plot_boxes(frame, name, centers, dim):
    y_max, x_max, _  = dim
    
    for i in range(len(centers)):
        x, y = centers[i]
        x, y = np.int(x), np.int(y_max - y)
        half_size = 50
        top_left = x - half_size, y - half_size
        bot_right = x + half_size, y + half_size
        frame = cv2.rectangle(frame, top_left, bot_right, (0, 255, 0),  1)
        # frame = cv2.circle(frame, (x, y), 50, (0, 255, 0),  1)
    cv2.imshow(name, frame)

In [14]:
def orb_func_boxed(ret, frame):
    kp, frame = get_orb_kp(frame, orb=cv2.ORB_create(nfeatures=1000))
    dim = frame.shape
    Z = kp_to_pairs(kp, dim)
    centers = get_centers(Z, num=5)
    plot_boxes(frame, 'frame', centers, dim)
    
run(orb_func_boxed)